# T2* Mapping - Multi-echo FLASH

T2* mapping with a multi-echo FLASH sequence.

### Imports

In [ ]:
import tempfile
from pathlib import Path

import matplotlib.pyplot as plt
import MRzeroCore as mr0
import numpy as np
import torch
from cmap import Colormap
from einops import rearrange
from mrpro.algorithms.reconstruction import DirectReconstruction
from mrpro.data import KData
from mrpro.data.traj_calculators import KTrajectoryIsmrmrd
from mrpro.operators import DictionaryMatchOp
from mrpro.operators.models import MonoExponentialDecay

from mrseq.scripts.t2star_multi_echo_flash import main as create_seq
from mrseq.utils import combine_ismrmrd_files
from mrseq.utils import sys_defaults

### Settings
We are going to use a numerical phantom with a matrix size of 128 x 128. 
To ensure we really acquire all radial lines in the steaady state, we use a large number of dummy excitations before the actual image acquisition. 

In [ ]:
image_matrix_size = [128, 128]

tmp = tempfile.TemporaryDirectory()
fname_mrd = Path(tmp.name) / 't2star.mrd'

### Create the digital phantom

We use the standard Brainweb phantom from [MRzero](https://github.com/MRsources/MRzero-Core), but we choose the B1-field to be constant everywhere.

In [ ]:
phantom = mr0.util.load_phantom(image_matrix_size)
phantom.B1[:] = 1.0

### Create the radial FLASH sequence

To create the radial FLASH sequence, we use the previously imported [radial_flash script](../src/mrseq/scripts/radial_flash.py).

In [ ]:
sequence, fname_seq = create_seq(
    system=sys_defaults,
    test_report=False,
    timing_check=False,
    fov_xy=float(phantom.size.numpy()[0]),
    n_readout=image_matrix_size[0],
    n_echoes=8,
    acceleration=1,
)

### Simulate the sequence

Now, we pass the sequence and the phantom to the MRzero simulation and save the simulated signal as an (ISMR)MRD file.

In [ ]:
mr0_sequence = mr0.Sequence.import_file(str(fname_seq.with_suffix('.seq')))
signal, ktraj_adc = mr0.util.simulate(mr0_sequence, phantom, accuracy=1e-1)
mr0.sig_to_mrd(fname_mrd, signal, sequence)
combine_ismrmrd_files(fname_mrd, Path(f'{fname_seq}_header.h5'))

### Reconstruct the image
We use [MRpro](https://github.com/PTB-MR/MRpro) for the image reconstruction.

In [ ]:
kdata = KData.from_file(str(fname_mrd).replace('.mrd', '_with_traj.mrd'), trajectory=KTrajectoryIsmrmrd())
recon = DirectReconstruction(kdata, csm=None)
idata = recon(kdata)

### Compare to theoretical model

We can now compare the result to a simulation using the idealized signal model for spoiled gradient echo sequences. First, we need to calculate $T2^*$ using $1/T2^* = 1/T2 + 1/T2'$.

In [ ]:
t2star = 1 / (1 / phantom.T2 + 1 / phantom.T2dash)

In [ ]:
dictionary = DictionaryMatchOp(MonoExponentialDecay(decay_time=idata.header.te), index_of_scaling_parameter=0)
dictionary.append(torch.tensor(1.0), torch.linspace(0.01, 0.8, 1000)[None, :])
m0_match, t2star_match = dictionary(idata.data[:, 0, 0])

t2star_input = np.roll(rearrange(t2star.numpy().squeeze()[::-1, ::-1], 'x y -> y x'), shift=(1, 1), axis=(0, 1))
obj_mask = np.zeros_like(t2star_input)
obj_mask[t2star_input > 0] = 1
t2star_measured = t2star_match.numpy().squeeze() * obj_mask

fig, ax = plt.subplots(1, 3, figsize=(15, 3))
for cax in ax:
    cax.set_xticks([])
    cax.set_yticks([])

im = ax[0].imshow(t2star_input, vmin=0, vmax=0.08, cmap=Colormap('navia').to_mpl())
fig.colorbar(im, ax=ax[0], label='Input T2* (s)')

im = ax[1].imshow(t2star_measured, vmin=0, vmax=0.08, cmap=Colormap('navia').to_mpl())
fig.colorbar(im, ax=ax[1], label='Measured T2* (s)')

im = ax[2].imshow(t2star_measured - t2star_input, vmin=-0.008, vmax=0.008, cmap='bwr')
fig.colorbar(im, ax=ax[2], label='Difference T2* (s)')

relative_error = np.sum(np.abs(t2star_input - t2star_measured)) / np.sum(np.abs(t2star_input))
print(f'Relative error {relative_error}')
assert relative_error < 0.02